In [1]:
#Monta il Drive di Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import random
import torchvision
import numpy as np
import torch

import matplotlib
import matplotlib.pyplot as plt

import json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilies
from detectron2 import model_zoo
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer, SimpleTrainer
from detectron2.config import get_cfg, CfgNode
from detectron2.data.detection_utils import read_image, annotations_to_instances, filter_empty_instances,transform_instance_annotations

# DataAugumentation
from detectron2.data import build_detection_train_loader
from detectron2.data import DatasetMapper
from detectron2.data import transforms as T
from PIL import Image

# Evaluation
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

import time
# Root directory of the project
ROOT_DIR = os.getcwd()

In [3]:
class CocoTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(dataset_name, cfg, True, output_folder)

In [4]:
#DATA ARGUMENTATION
def custom_mapper(single_example_dict):
    single_example_dict_aug = single_example_dict.copy() # it will be modified by code below
    image = read_image(single_example_dict["file_name"], format="BGR")
    transform_list = [
                      #T.Resize((480, 640)),
                      T.RandomBrightness(0.5, 1.1),
                      T.RandomContrast(0.8, 3),
                      T.RandomRotation([-20, 20], expand=False, interp=Image.LINEAR)
                      ]

    image, transforms = T.apply_transform_gens(transform_list, image)
    single_example_dict_aug["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32")) #transpose per allenamento su GPU

    annos = [
        transform_instance_annotations(obj, transforms, image.shape[:2])
        for obj in single_example_dict_aug.pop("annotations")
    ]
    instances = annotations_to_instances(annos, image.shape[:2])
    single_example_dict_aug["instances"] = filter_empty_instances(instances)
    return  single_example_dict_aug

In [5]:
class CustomTrainer(DefaultTrainer):
  @classmethod
  def build_train_loader(cls, cfg):
    return build_detection_train_loader(cfg, mapper=custom_mapper)

In [6]:
import os
class Config:
    def __init__(self) :
        # - Path
        self.path_data = '/content/drive/MyDrive/data'
        self.labeled_data = os.path.join(self.path_data, '/content/drive/MyDrive/data/labeled-data')
        self.name_train_dataset = "train"
        self.name_valid_dataset = "valid"
        self.name_test_dataset = "test"

        self.path_coco_json_train = os.path.join(self.path_data, 'train.json')
        self.path_coco_json_valid = os.path.join(self.path_data, 'valid.json')
        self.path_coco_json_test = os.path.join(self.path_data, 'test.json')
        self.path_output_dir = "/content/drive/MyDrive/data/outputs_segmentation"


        # Training
        self.EPOCH = 100 # In detectron2, epoch is MAX_ITER * BATCH_SIZE / TOTAL_NUM_IMAGES
        self.EPOCH_DECREASE_LR = 10
        self.EPOCH_CHECKPOINT = 1
        self.NUM_AUG = 3

In [7]:
def load_configuration(config):
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.DATASETS.TRAIN = ("/content/drive/MyDrive/data/train.json",)
    cfg.DATASETS.TEST = ("/content/drive/MyDrive/data/test.json",)
    cfg.DATALOADER.NUM_WORKERS = 2
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Use pre-trained weights
    cfg.SOLVER.IMS_PER_BATCH = 2
    cfg.SOLVER.BASE_LR = 0.0001
    cfg.SOLVER.MAX_ITER = 1    # Adjust to your needs
    cfg.SOLVER.GAMMA = 0.1
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # Set the number of classes
    cfg.MODEL.DEVICE = "cpu"
    cfg.SOLVER.OPTIMIZER = "ADAMW"
    cfg.SOLVER.BETAS = (0.9, 0.999)  # I parametri beta per l'ottimizzatore AdamW
    cfg.TEST.EVAL_PERIOD = 100  # Evaluate the model every 100 iterations
    cfg.VIS_PERIOD = 0  # Do not visualize the predictions
    cfg.OUTPUT_DIR = "./output"  # Directory where the output would be written
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.50   # set the testing threshold for this model
    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

    return cfg

In [8]:
def view_dataset(name_dataset):
  dataset = DatasetCatalog.get(name_dataset)
  metadata = MetadataCatalog.get(name_dataset)

  for dic in dataset:
    print(dic)
    immagine = cv2.imread(dic['file_name'])
    visualizer = Visualizer(immagine, metadata=metadata) #img deve essere in rgb
    vis = visualizer.draw_dataset_dict(dic)
    cv2_imshow(vis.get_image())

In [ ]:
config = Config()
cfg = load_configuration(config)
view_dataset(cfg.DATASETS.TRAIN )

In [10]:
def view_prediction(predictor, name_dataset):
      dataset = DatasetCatalog.get(name_dataset)
      metadata = MetadataCatalog.get(name_dataset)

      for dic in dataset:
          print(dic)
          img = cv2.imread(dic["file_name"])
          outputs = predictor(img)
          visualizer = Visualizer(img, metadata=metadata, scale=0.8)
          vis = visualizer.draw_instance_predictions(outputs["instances"].to("cpu")) #Passing the predictions to CPU from the GPU
          image = vis.get_image()
          cv2.imshow("Image", image)
          cv2.waitKey(0)
          cv2.destroyAllWindows()
          id = dic["file_name"].split("/")[-1]
          cv2.imwrite(id, vis.get_image())

          img = cv2.cvtColor(vis.get_image()[:, :, ::-1], cv2.COLOR_RGBA2RGB)
          plt.imsave(os.path.join(os.path.join(cfg.OUTPUT_DIR), str(id) + '.jpg'), img) #save predictions

In [11]:
def testing(cfg):
    # Specifica il percorso completo della cartella in cui sono stati salvati i pesi
    folder_path = '/content/drive/MyDrive/data'
    # Specifica il percorso completo del file di checkpoint
    checkpoint_path = os.path.join(folder_path,'pesi_modello_dopo_train.h5')
    # Verifica se il file di checkpoint esiste nel percorso specificato
    if os.path.exists(checkpoint_path):
        # Carica i pesi del modello dal file di checkpoint
        cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(checkpoint_path)
        print(cfg.MODEL.WEIGHTS)
    else: print("Il file di checkpoint non esiste nel percorso specificato.")
    cfg.DATASETS.TEST = ("/content/drive/MyDrive/data/test.json",)
    predictor = DefaultPredictor(cfg)
    return view_prediction(predictor,name_dataset=cfg.DATASETS.TEST)

In [16]:
def evaluate(cfg):
    cfg.MODEL.DEVICE = "cuda"
    folder_path = '/content/drive/MyDrive/data'
    checkpoint_path = os.path.join(folder_path,'pesi_modello_dopo_train.h5')
    if os.path.exists(checkpoint_path):
        cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(checkpoint_path)
        print("Pesi del modello dopo il train ripresi con successo: \n")
        print(cfg.MODEL.WEIGHTS)
    else:print("Il file di checkpoint non esiste nel percorso specificato.")
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9  # set the testing threshold for this model
    cfg.TEST.KEYPOINT_OKS_SIGMAS = [] # per il calcolo dell OKS
    #Pass the validation dataset
    cfg.DATASETS.TEST = ("/content/drive/MyDrive/data/test.json",)
    #Call the COCO Evaluator function and pass the Validation Dataset
    evaluator = COCOEvaluator(cfg.DATASETS.TEST, cfg, distributed=False, output_dir=cfg.OUTPUT_DIR)
    val_loader = build_detection_test_loader(cfg, cfg.DATASETS.TEST)
    #Use the created predicted model in the previous step
    predictor = DefaultPredictor(cfg)
    inference_on_dataset(predictor.model, val_loader, evaluator)

In [17]:
def evaluate_dice_coefficent(cfg):
      cfg= load_configuration()
      ###########################################
      #      symmetric Unified Focal loss      #
      ###########################################
      def sym_unified_focal_loss(weight=0.5, delta=0.6, gamma=0.5):
          def loss_function(y_true,y_pred):
            symmetric_ftl = symmetric_focal_tversky_loss(delta=delta, gamma=gamma)(y_true,y_pred)
            symmetric_fl = symmetric_focal_loss(delta=delta, gamma=gamma)(y_true,y_pred)
            if weight is not None:
              return (weight * symmetric_ftl) + ((1-weight) * symmetric_fl)
            else:
              return symmetric_ftl + symmetric_fl
          return loss_function
      ###########################################
      #      Asymmetric Unified Focal loss      #
      ###########################################
      def asym_unified_focal_loss(weight=0.5, delta=0.6, gamma=0.5):
        def loss_function(y_true,y_pred):
          asymmetric_ftl = asymmetric_focal_tversky_loss(delta=delta, gamma=gamma)(y_true,y_pred)
          asymmetric_fl = asymmetric_focal_loss(delta=delta, gamma=gamma)(y_true,y_pred)
          if weight is not None:
            return (weight * asymmetric_ftl) + ((1-weight) * asymmetric_fl)
          else:
            return asymmetric_ftl + asymmetric_fl

        return loss_function
      ###########################################
      #      Dice Focal loss      #
      ###########################################

      def dice_coefficent(y_true, y_pred, smooth=1):
        intersection = np.sum(y_true * y_pred) # TP
        union = np.sum(y_true) + np.sum(y_pred)  #2TP + FP + FN Nota: il 2TP è dovuto al fatto che l'aria di intersezione
        #la consideriamo 2 volte: (i) per il y_true e (ii) per il y_pred
        dice = (2. * intersection + smooth)/(union + smooth)
        return dice

      cfg.MODEL.DEVICE = "cpu"

      #THRESH???
      cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
      cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.50  # set the testing threshold for this model
      cfg.TEST.KEYPOINT_OKS_SIGMAS = [] # per il calcolo dell OKS
      #Pass the validation dataset
      cfg.DATASETS.TEST = ("my_dataset_test")

      dataset = DatasetCatalog.get(cfg.DATASETS.TEST)
      predictor = DefaultPredictor(cfg)
      tot_dice = []
      cnt_no_pred = 0
      cnt_pred = 0
      for dic in dataset:
          immagine = cv2.imread(dic['file_name'])
          pred = predictor(immagine)


          # Pred mask
          pred_mask = pred['instances'].get_fields()['pred_masks']

          #(pred['instances'].get_fields()['pred_masks'])
          pred_mask = pred_mask.numpy()
          if pred_mask.shape[0] == 0:
            pred_mask = np.zeros((480,640,1))
            cnt_no_pred += 1
            continue
          else:
            print(cnt_pred, ' #####')
            print(dic['file_name'])
            cnt_pred += 1
            pred_mask = pred_mask * 1
            pred_mask = np.transpose(pred_mask, [1,2,0])
            # Target Mask
            annotations = dic['annotations'][0]['segmentation'][0]
            img = Image.new('L', (640,480), 0)
            ImageDraw.Draw(img).polygon(annotations, outline=0, fill=1)
            mask = np.array(img).reshape(480,640,1)
            dice_value = dice_coefficent(mask,pred_mask)
            loss_value_1 = sym_unified_focal_loss(mask,pred_mask)
            loss_value_2= asym_unified_focal_loss(mask,pred_mask)
            tot_dice.append(float(dice_value))
            dice_value = dice_coefficent(mask,pred_mask)
            loss_value_1 = sym_unified_focal_loss(mask,pred_mask)
            loss_value_2= asym_unified_focal_loss(mask,pred_mask)
            tot_dice1.append(float(loss_value_1))#faccio solo quella simmetrica
            tot_dice2.append(float(loss_value_2))#faccio solo quella asimmetrica
            tot_dice.append(float(dic))#faccio solo quella simmetrica
            print('\n')

            print("Tot imgs: ", cnt_pred + cnt_no_pred)
            print("Predette imgs: ", cnt_pred)
            print("No Predette imgs: ", cnt_no_pred)
            print("Dice_loss",np.argmin(np.array(tot_dice)))
            print("Simmetric_loss",np.argmin(np.array(tot_dice1)))
            print("Assimetric_loss",np.argmin(np.array(tot_dice2)))

In [18]:
### Load configuration
config = Config()
cfg = load_configuration(config)

In [19]:
if not os.path.exists(cfg.OUTPUT_DIR):
  os.makedirs(cfg.OUTPUT_DIR)

print("Num ITER tot: ", cfg.SOLVER.MAX_ITER )
print("LR ITER update: ", cfg.SOLVER.STEPS )
print("Save CHECKPOINT: ", cfg.SOLVER.CHECKPOINT_PERIOD)
print(cfg.MODEL.WEIGHTS)

Num ITER tot:  1
LR ITER update:  (210000, 250000)
Save CHECKPOINT:  5000
https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl


In [45]:
def train(cfg):
    if not os.path.exists(cfg.OUTPUT_DIR):
          os.makedirs(cfg.OUTPUT_DIR)
    trainer = CocoTrainer(cfg)
    trainer.resume_or_load(resume=False)
    trainer.train()
    # Specifica il percorso completo della cartella di destinazione
    folder_path = '/content/drive/MyDrive/data'
    # Assicurati che la cartella esista
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    # Specifica il percorso completo del file di checkpoint
    checkpoint_path = os.path.join(folder_path, 'pesi_modello_dopo_train.h5')
    # Salva i pesi del modello nel percorso specificato
    trainer.save_weights(checkpoint_path.format(epoch=0))
    print("I Pesi della Rete sono stati salvati correttamente")

In [25]:
from detectron2.data.datasets import register_coco_instances

In [42]:
def register_dataset(path_json, path_data, name_dataset):
    with open(path_json) as f:
        json_file = json.load(f)
    register_coco_instances(name_dataset,  # name (str): the name that identifies a dataset, e.g. "coco_2014_train".
                            {},
                            path_json,  # json_file (str): path to the json instance annotation file.
                            path_data # image_root (str or path-like): directory which contains all the images.
                            )

In [44]:
if __name__ == "__main__":
    config = Config()
    name_dataset_train = "TRAIN"
    name_dataset_valid = "VALID"
    name_dataset_test = "TEST"

    ### REGISTRAZIONE DATASET (va fatto solo la prima volta)
    register_dataset(config.path_coco_json_train, config.path_data, name_dataset_train)
    register_dataset(config.path_coco_json_valid, config.path_data, name_dataset_valid)
    register_dataset(config.path_coco_json_test, config.path_data, name_dataset_test)

In [46]:
config = Config()
cfg = load_configuration(config)
train(cfg)

[06/21 15:53:29 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

KeyError: ignored

In [1]:
testing(cfg)

NameError: ignored